In [7]:
def get_web_data(dom=None, cookies=None, header=None):
    '''
    获取每页评论数据
    '''
    # //这里可以表示多个目录，当有多个目录时可以用//来代替，@class="comment-item"表示标签下的特定属性，这里用于定位特定的div标签，text()表示标签的文本信息
    names = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/a/text()')           # 用户名
    print(names)
    ratings = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/span[2]/@class')   # 评分
    times = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/span[@class="comment-time "]/@title')     # 评论发布时间
    message = dom.xpath('//div[@class="comment-item"]//div[@class="comment"]//span[@class="short"]/text()')    # 短评正文
    user_url = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/a/@href')           # 用户主页网址
    votes = dom.xpath('//div[@class="comment-item"]//div[@class="comment"]//span[@class="votes"]/text()')    # 赞同数量
    cities = []
    load_times = []
    for i in user_url:
        web_data = requests.get(i, cookies=cookies, headers=header)
        dom_url = etree.HTML(web_data.text, etree.HTMLParser(encoding='utf-8'))
        address = dom_url.xpath('//div[@class="basic-info"]//div[@class="user-info"]/a/text()')                    # 用户居住地
        load_time = dom_url.xpath('//div[@class="basic-info"]//div[@class="user-info"]/div[@class="pl"]/text()')   # 用户入会时间
        cities.append(address)
        load_times.append(load_time)
        print(load_times)
        time.sleep(5)  # 设置休息时间，避免被反爬虫
    # 来获取具体评分中的数字，由于有的用户没用评分所以返回空,findall返回的是一个列表，\d可以用[0-9]替换
    ratings = ['' if 'rating' not in i else int(re.findall('\d{2}', i)[0]) for i in ratings]   # 评分数据整理
    load_times = ['' if i == [] else i[1].strip()[:-2] for i in load_times]   # 入会数据整理
    cities = ['' if i == [] else i[0] for i in cities]   # 居住地数据整理
    data = pd.DataFrame({
        '用户名': names,
        '居住城市': cities,
        '加入时间': load_times,
        '评分': ratings,
        '发表时间': times,
        '短评正文': message,
        '赞同数量': votes
    })
    return data

from selenium import webdriver
from lxml import etree
import requests
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

driver = webdriver.Chrome()   # 启动chrome浏览器
url = 'https://movie.douban.com/subject/26266893/comments?status=P'
driver.get(url)   # 获取网页源码数据

# 通过cookies来进行登录操作
cookies = {}
cookies_str = 'bid=gRZz1WM1-Zs; douban-fav-remind=1; __utma=30149280.1144612666.1588138806.1591007409.1591195137.4; __utmz=30149280.1591195137.4.4.utmcsr=cn.bing.com|utmccn=(referral)|utmcmd=referral|utmcct=/; ap_v=0,6.0; _pk_ses.100001.4cf6=*; dbcl2="219869902:Xw87UAj00d8"; ck=epfK; _pk_id.100001.4cf6=bcd3d505e972e418.1595576154.3.1595580708.1595578108.; push_noty_num=0; push_doumail_num=0'
for i in cookies_str.split(';'):  # 将cookies整理成所需格式，这里整理为字典格式
    k, v = i.split('=', 1)  # 1表示只按照第一个等号进行拆分，后面如果还有等号则不进行拆分
    cookies[k] = v

# 对所有页面进行数据爬取及解析操作，并进行数据保存
all_data = pd.DataFrame()  # 用于将所有清洗好的数据加载到数据框内(每清洗一页数据就加载一夜数据)
wait = WebDriverWait(driver, 5)  # 设置网页加载等待时间，用于实现如果在规定的时间内网页没有加载进来则报错
# 由于要爬取的网站有反爬虫的机制，所以要添加header属性，不添加会返回418
header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}
while True:
    
    wait.until(  # 这里用网页加载等待时间来判定一个网页是否加载进来
        EC.element_to_be_clickable(   # 通过该项条件确认网页是否已经加载进来,防止断网导致加载不成功
            (By.CSS_SELECTOR, '#comments > div:nth-child(20) > div.comment > h3 > span.comment-info > a') # 当前页最后一个“用户”按钮是否可以点击
        )  # 如果在规定的时间内不能加载网页则终止循环操作
    )
    
    dom = etree.HTML(driver.page_source, etree.HTMLParser(encoding='utf-8'))   # 网页源码解析，得到一个dom文件
    data = get_web_data(dom=dom, cookies=cookies, header=header)
    all_data = pd.concat([all_data, data], axis=0)  # 将每一页的获取到的数据进行拼接
    # 在评论的最后一页中下一页按钮是无法点击的，同时在源码中没有selector，所以通过查找来决定循环的跳出
    if driver.find_element_by_css_selector('#paginator > a.next')==[]:   # 判定是否还有“后页”按钮
        break  
    
    confirm_bnt = wait.until(  # 点击按钮是否加载且是否可以被点击
        EC.element_to_be_clickable(
            (By.CSS_SELECTOR, '#paginator > a.next')
        )
    )
    confirm_bnt.click()   # 执行翻页操作
    
all_data.to_csv('douban01.csv', index=None, encoding='GB18030')   # 将数据以csv文件的方式写出

all_data.shape



['陆支羽', '乌鸦火堂', 'frozenmoon', '江寒园', '张小北', '舒服Kimodi', '姨妈的鸭', '桃桃林林', '九只苍蝇撞墙', '巽', '影志', '王小叶儿', '沙雕电影', '凌睿', '亵渎电影', '刘小流', '小姬', '二十二岛主', '顿河', '乙小囧']
[['luzhiyu ', ' 2008-08-30加入']]
[['luzhiyu ', ' 2008-08-30加入'], ['dreamfox ', ' 2008-03-12加入']]
[['luzhiyu ', ' 2008-08-30加入'], ['dreamfox ', ' 2008-03-12加入'], ['1233038 ', ' 2006-08-12加入']]
[['luzhiyu ', ' 2008-08-30加入'], ['dreamfox ', ' 2008-03-12加入'], ['1233038 ', ' 2006-08-12加入'], ['63306399 ', ' 2012-07-22加入']]
[['luzhiyu ', ' 2008-08-30加入'], ['dreamfox ', ' 2008-03-12加入'], ['1233038 ', ' 2006-08-12加入'], ['63306399 ', ' 2012-07-22加入'], ['xzfd ', ' 2008-03-04加入']]
[['luzhiyu ', ' 2008-08-30加入'], ['dreamfox ', ' 2008-03-12加入'], ['1233038 ', ' 2006-08-12加入'], ['63306399 ', ' 2012-07-22加入'], ['xzfd ', ' 2008-03-04加入'], ['58713868 ', ' 2012-02-19加入']]
[['luzhiyu ', ' 2008-08-30加入'], ['dreamfox ', ' 2008-03-12加入'], ['1233038 ', ' 2006-08-12加入'], ['63306399 ', ' 2012-07-22加入'], ['xzfd ', ' 2008-03-04加入'], ['58713868 ', ' 2012-02-19加

[['yyy18n ', ' 2009-12-20加入'], ['reave ', ' 2007-08-10加入'], ['darkwood ', ' 2006-07-22加入'], ['168391468 ', ' 2017-10-19加入'], [], ['phoenix43 ', ' 2009-03-08加入'], ['hexiaoqin ', ' 2009-09-11加入'], ['1892627 ', ' 2007-10-05加入'], ['63469119 ', ' 2012-07-29加入'], ['3540441 ', ' 2009-01-25加入']]
[['yyy18n ', ' 2009-12-20加入'], ['reave ', ' 2007-08-10加入'], ['darkwood ', ' 2006-07-22加入'], ['168391468 ', ' 2017-10-19加入'], [], ['phoenix43 ', ' 2009-03-08加入'], ['hexiaoqin ', ' 2009-09-11加入'], ['1892627 ', ' 2007-10-05加入'], ['63469119 ', ' 2012-07-29加入'], ['3540441 ', ' 2009-01-25加入'], ['Uvo ', ' 2005-10-03加入']]
[['yyy18n ', ' 2009-12-20加入'], ['reave ', ' 2007-08-10加入'], ['darkwood ', ' 2006-07-22加入'], ['168391468 ', ' 2017-10-19加入'], [], ['phoenix43 ', ' 2009-03-08加入'], ['hexiaoqin ', ' 2009-09-11加入'], ['1892627 ', ' 2007-10-05加入'], ['63469119 ', ' 2012-07-29加入'], ['3540441 ', ' 2009-01-25加入'], ['Uvo ', ' 2005-10-03加入'], ['diewithme ', ' 2009-07-18加入']]
[['yyy18n ', ' 2009-12-20加入'], ['reave ', ' 20

[['timonin ', ' 2011-12-20加入'], ['159673853 ', ' 2017-03-29加入'], ['flowermumu ', ' 2006-04-21加入'], ['2849243 ', ' 2008-08-25加入'], ['88345829 ', ' 2014-05-05加入'], ['longzheking ', ' 2009-05-28加入'], ['zh6289 ', ' 2010-05-09加入'], ['48554953 ', ' 2011-01-03加入'], ['dyingpluto ', ' 2006-06-19加入'], ['misterinfamous ', ' 2011-03-07加入'], ['graycat ', ' 2010-01-22加入'], ['ydyie ', ' 2007-12-26加入'], ['luhua ', ' 2006-11-27加入'], ['2049622 ', ' 2007-11-30加入'], ['pajamania ', ' 2010-05-03加入']]
[['timonin ', ' 2011-12-20加入'], ['159673853 ', ' 2017-03-29加入'], ['flowermumu ', ' 2006-04-21加入'], ['2849243 ', ' 2008-08-25加入'], ['88345829 ', ' 2014-05-05加入'], ['longzheking ', ' 2009-05-28加入'], ['zh6289 ', ' 2010-05-09加入'], ['48554953 ', ' 2011-01-03加入'], ['dyingpluto ', ' 2006-06-19加入'], ['misterinfamous ', ' 2011-03-07加入'], ['graycat ', ' 2010-01-22加入'], ['ydyie ', ' 2007-12-26加入'], ['luhua ', ' 2006-11-27加入'], ['2049622 ', ' 2007-11-30加入'], ['pajamania ', ' 2010-05-03加入'], ['emilycat ', ' 2007-06-26加入']]


KeyboardInterrupt: 

In [8]:
all_data

,用户名,居住城市,加入时间,评分,发表时间,短评正文,赞同数量
0,陆支羽,北京,2008-08-30,50,2019-01-29 20:10:48,1.终于，轮到我们仰望星空。2.后启示录死亡废墟，赛博朋克地下城，以及烟波浩渺的末日想象，缔...,62247
1,乌鸦火堂,北京,2008-03-12,50,2019-01-20 19:00:54,华语真正意义上的第一部科幻大片！刘慈欣的硬核科幻设定，小兵扛大旗的主流价值观人设，东方式的家...,15026
2,frozenmoon,北京,2006-08-12,30,2019-02-05 16:29:48,1.从特效和技术上讲，这应该是迄今为止此类中国电影的巅峰了。磅礴恢宏，细节营造用心。2.故事...,4398
3,江寒园,陕西西安,2012-07-22,30,2019-02-05 14:31:49,1. 煽情、尴尬与生硬并存\n2.主要是2001太空漫游（致敬AI杀人太空戏）和罗兰.艾默里...,3928
4,张小北,北京,2008-03-04,50,2019-01-29 02:11:13,从各个方面来说都是一部好看的类型片。而最特别的是它是一部国产科幻电影，影迷+科幻迷的双重满足...,12909
5,舒服Kimodi,辽宁大连,2012-02-19,50,2019-02-06 03:54:42,8.8，热评第一条我就呵呵了，中国科幻电影为什么不值得我们打高分去鼓励，非要放在国际水平线上...,9644
6,姨妈的鸭,上海,2009-12-01,50,2019-02-05 16:51:54,春节期间最大的一场烟花，是看你爸为你和地球点燃的宇宙爆炸，太朋克了。,6718
7,桃桃林林,"P'yŏngyang, North Korea",2005-10-09,30,2019-02-07 23:08:46,7分，当时看的时候，最大的感受就是，影片终于用一种好莱坞灾难片的模式，来处理这部电影，这使得...,1602
8,九只苍蝇撞墙,"Montréal, Canada",2007-01-11,,2019-02-05 23:16:40,战天斗地会是历史留给中国人的唯一思想遗产么,1920
9,巽,,2007-12-07,50,2019-01-22 05:43:32,4星，多一星鼓励。华语电影工业飞跃的节点，技术、剧本、精神内核，各方面都是全球尺度下顶级同行...,5032
